In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier
# import lightgbm as lgb
# import xgboost as xgb

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dataset = pd.read_csv('csv/dataset.csv')
dataset = dataset[dataset['Квалификация'] != 3]
dataset = dataset[dataset['Квалификация'] != 4]
data = dataset.iloc[:, :-1]
target = dataset.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
clf = DummyClassifier(strategy='most_frequent')
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print("Число ошибочно классифицированных образцов DummyClassifier: % d" % (y_test != y_pred).sum())
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_name))

print()
print('Логик регрессион')
print('Вероятность моего отчисления за первый семестр:',clf.predict(pd.read_csv('uspevaemost_nakop_My.csv').iloc[:1,:])[:, 1] * 100)
print('Вероятность моего отчисления за второй семестр:',clf.predict(pd.read_csv('uspevaemost_nakop_My.csv').iloc[1:2,:])[:, 1] * 100)
print('Вероятность моего отчисления за третий семестр:',clf.predict(pd.read_csv('uspevaemost_nakop_My.csv').iloc[2:3,:])[:, 1] * 100)
print('Вероятность моего отчисления за четвертый семестр:',clf.predict(pd.read_csv('uspevaemost_nakop_My.csv').iloc[3:4,:])[:, 1] * 100)
print()

In [ ]:
lr = LogisticRegression(C=1000.0, max_iter=1000000000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

target_name = ['0 - не отчислен', 'Отчислен']
print(classification_report(y_test, y_pred, target_names=target_name))
print(accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

# print('Логик регрессион')
# print('Вероятность моего отчисления за первый семестр:',lr.predict_proba(pd.read_csv('csv/uspevaemost_nakop_My.csv').iloc[:1,:])[:, 1] * 100)
# print('Вероятность моего отчисления за второй семестр:',lr.predict_proba(pd.read_csv('csv/uspevaemost_nakop_My.csv').iloc[1:2,:])[:, 1] * 100)
# print('Вероятность моего отчисления за третий семестр:',lr.predict_proba(pd.read_csv('csv/uspevaemost_nakop_My.csv').iloc[2:3,:])[:, 1] * 100)
# print('Вероятность моего отчисления за четвертый семестр:',lr.predict_proba(pd.read_csv('csv/uspevaemost_nakop_My.csv').iloc[3:4,:])[:, 1] * 100)


In [27]:
best = np.zeros([3])



for i in range(1, 15):
        for j in range(1, 3):
            model = CatBoostClassifier(iterations=1000, devices='0:1', depth=i,        # Глубина деревьев
                                    learning_rate=1/(10**j),  # Скорость обучения
                                    loss_function='Logloss',  # Функция потерь для задачи классификации
                                    logging_level='Silent')
            model.fit(X_train_std, y_train)

            y_pred = model.predict(X_test_std)

            accuracy = accuracy_score(y_test, y_pred)
            report = classification_report(y_test, y_pred)

            info = np.array([f'Глубина {i}', f'Скорость обучения {1/(10**j)}', accuracy])
            print(info)
            best = np.vstack((best, info))

max_row_index = np.argmax(best[:, 2])

# Extract the row with the maximum value
max_row = best[max_row_index, :]

print("Row with the maximum value in column {}: {}".format(2, max_row))


In [ ]:
model = CatBoostClassifier(iterations=1000, task_type="GPU", devices='0:1', depth=8,        # Глубина деревьев
                                    learning_rate=0.1,  # Скорость обучения
                                    loss_function='Logloss',  # Функция потерь для задачи классификации
                                    logging_level='Silent')
model.fit(X_train_std, y_train)

y_pred = model.predict(X_test_std)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(accuracy)

print(report)

In [ ]:
confusion_matrix(y_test, y_pred)


In [ ]:
sum(y_pred == 1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(X_train_std, y_train)

# Make predictions
tree_predictions = tree_classifier.predict(X_test_std)

# Evaluate the model
tree_accuracy = accuracy_score(y_test, tree_predictions)
print(f"Decision Tree Accuracy: {tree_accuracy}")

report = classification_report(y_test,  tree_predictions)
print(report)
confusion_matrix(y_test, tree_predictions)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train a random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_classifier.predict(X_test)

# Evaluate the model
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")
report = classification_report(y_test, y_pred)
print(report)
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.svm import SVC

# Create and train an SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_std, y_train)

# Make predictions
svm_predictions = svm_classifier.predict(X_test_std)

# Evaluate the model
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy}")
report = classification_report(y_test, svm_predictions)
print(report)
confusion_matrix(y_test, svm_predictions)

In [ ]:
from sklearn.naive_bayes import GaussianNB


    

# Create and train a Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_std, y_train)

# Make predictions
nb_predictions = nb_classifier.predict(X_test_std)

# Evaluate the model
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f"Naive Bayes Accuracy: {nb_accuracy}")
report = classification_report(y_test, nb_predictions)
print(report)
confusion_matrix(y_test, nb_predictions)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create and train a KNN classifier with k=5 (you can adjust k)
for i in range(1, 10):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    knn_classifier.fit(X_train_std, y_train)

    # Make predictions
    knn_predictions = knn_classifier.predict(X_test_std)

    # Evaluate the model
    knn_accuracy = accuracy_score(y_test, knn_predictions)
    print(f"KNN Accuracy: {knn_accuracy}")
    report = classification_report(y_test, knn_predictions)
    print(report)
    confusion_matrix(y_test, knn_predictions)

Row with the maximum value in column 2: ['Глубина 10' 'Скорость обучения 0.01' '0.9127774864534173']

In [ ]:
import numpy as np

# Create an initial NumPy array
arr = np.array([1, 2, 3])

# Append a single element


# Append multiple elements (as a list)
new_elements = [5, 6, 7]
arr = np.vstack((arr, new_elements))

print(arr)

In [ ]:
train_data = lgb.Dataset(X_train_std, label=y_train)
test_data = lgb.Dataset(X_test_std, label=y_test, reference=train_data)

# Определяем параметры модели
params = {
    'objective': 'binary',  # для задачи бинарной классификации
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',  # метрика качества
    'num_leaves': 50,
    'learning_rate': 0.01,
    'feature_fraction': 0.9
}

# Обучаем модель
num_round = 1000  # количество итераций обучения
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Предсказываем на тестовом наборе данных
y_pred = bst.predict(X_test_std, num_iteration=bst.best_iteration)

# Оцениваем качество модели
y_pred_binary = np.round(y_pred)  # преобразуем вероятности в бинарные предсказания
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)
confusion_matrix(y_test, y_pred_binary)

In [ ]:
params = {
    'objective': 'binary:logistic',  # для задачи бинарной классификации
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'eval_metric': 'logloss'
}

# Создаем объект DMatrix для данных
train_data = xgb.DMatrix(X_train_std, label=y_train)
test_data = xgb.DMatrix(X_test_std, label=y_test)

# Обучаем модель с ранней остановкой (early stopping)
watchlist = [(test_data, 'eval'), (train_data, 'train')]
num_round = 1000  # большое количество итераций
bst = xgb.train(params, train_data, num_round, evals=watchlist, early_stopping_rounds=10)

# Предсказываем на тестовом наборе данных
y_pred = bst.predict(test_data)

# Оцениваем качество модели
y_pred_binary = np.round(y_pred)  # преобразуем вероятности в бинарные предсказания
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)
confusion_matrix(y_test, y_pred_binary)

In [ ]:
y_test[y_test == 1].shape

In [ ]:
sum(y_pred == 1)

In [ ]:
for column, coef in zip(data.columns, lr.coef_[0]):
    print(f'Важность признака {column} равна {coef}')
